# Clase 22

Para una mejor visualización entrar al siguiente [link](https://nbviewer.jupyter.org/github/racsosabe/Miscelanea/blob/master/UPC/Clase%2022%20-%20Grafos%20X.ipynb)

# Requisitos Previos

* Matemática Básica
* Matemática Discreta
* Grafos

# Minimum Spanning Trees

## Spanning Tree

Un árbol de expansión o *spanning tree* de un grafo no dirigido y conexo $G = (V, E)$ es un subgrafo $T = (V, E')$ tal que:

1. $E' \subseteq E $.

2. $T$ es un árbol, por lo que es conexo y acíclico.

Si el grafo $G$ es ponderado con función de peso $w : E \to \mathbb{R}$, entonces el subgrafo $T$ también será ponderado y sus aristas tendrán el mismo peso que el que tenían en $G$. De esta forma, definimos el **peso** del árbol de expansión $T$ como:

$$ w(T) = \sum\limits_{(u, v) \in E'}w(u,v) $$

En términos generales, podemos considerar que el spanning tree es una forma de eliminar algunas aristas hasta que no hayan ciclos bajo la condición de que el grafo no debe dejar de ser conexo.

## Calculando el Minimum Spanning Tree

Dado un grafo no dirigido, conexo y ponderado $G = (V, E)$ con función de peso $w : E \to \mathbb{R}$, definimos un **Minimum Spanning Tree (MST) $T$** como **cualquier árbol de expansión de $G$** tal que no exista algún otro árbol de expansión con peso menor que el de $T$.

Esta definición es bastante clara, pero el problema ahora radica en cómo obtener el MST de manera eficiente: Plantearemos un método genérico y probaremos que dicho método es válido para obtener un MST.

Usaremos de referencia un conjunto de aristas $A \subseteq E$, el cual cumplirá con la invariante

- Antes de cada iteración, $A$ es un subconjunto de las aristas de algún MST.

De esta forma, en cada iteración nos aseguraremos de agregar a $A$ una arista $(u, v)$ tal que $A \cup \{(u, v)\}$ no viole la invariante. A la arista $(u, v)$ se le denomina **safe edge** para A, ya que es seguro agregarla al conjunto $A$.

```Python
MST-Generico()
    A = {}
    while A no es Spanning Tree:
        Encontrar una safe edge para A: (u, v)
        A = A U {(u, v)}
    return A
```

La prueba de la conservación de invariante se deja como ejercicio, no es complicada de realizar.

Notemos que el algoritmo anterior es simple, pero la dificultad principal está en encontrar una safe edge para $A$ de manera eficiente, lo cual discutiremos en los algoritmos de Prim y de Kruskal.

## Cortes

Definimos como **corte $(S, V \backslash S)$** de un grafo $G = (V, E)$ a una partición de $V$. Decimos que una arista $(u, v)$ **cruza** el corte $(S, V \backslash S)$ si uno de sus extremos está en $S$ y el otro en $V \backslash S$. 

Un corte **respeta** el conjunto de aristas $A$ si ninguna arista de $A$ cruza dicho corte. Una arista que cruza un corte es **ligera** si tiene el mínimo peso de entre todas las aristas que también lo cruzan. En general, una arista **ligera** respecto a alguna propiedad es aquella con el mínimo peso de entre todas las aristas que cumplen la propiedad.

**Teorema:** Sea un grafo no dirigido, conexo y ponderado $G = (V, E)$ con función de pesos $w : E \to \mathbb{R}$. Si $A$ es un subconjunto de $E$ tal que pertenece a algún MST de $G$ y sean $(S, V\backslash S)$ cualquier corte que respete a $A$ y $(u, v)$ una arista ligera que cruza al corte $(S, V \backslash S)$; entonces $(u, v)$ es una safe edge para $A$.

**Prueba:**

Sea $T$ un MST que incluya a $A$ y asumamos que $T$ no contiene la arista ligera $(u, v)$, ya que en el caso que lo hiciera, el teorema cumple. Deberemos construir un MST $T'$ tal que incluya a $A \cup \{(u, v)\}$. 

La arista $(u, v)$ forma un ciclo con las aristas del camino $p$ de $u$ a $v$ en $T$, dado que $u$ y $v$ están en conjuntos opuestos del corte $(S, V\backslash S)$, al menos una arista de $p$ cruza el corte, sea una de esas aristas $(x, y)$. Notemos que $(x, y) \not \in A$, ya que el corte respeta a $A$. El quitar esta arista $(x, y)$ de $T$ lo vuelve 2 componentes disjuntas tales que $u$ y $v$ pertenecen a componentes diferentes. El agregar $(u, v)$ a las componentes anteriores las vuelve un spanning tree. Entonces hemos construido $T' = T \backslash \{(x, y)\} \cup \{(u, v)\}$ como un spanning tree.

Ahora, ya que $(u, v)$ es una arista ligera y $(x, y)$ cruza el corte $(S, V \backslash S)$, se debe cumplir que:

$$ w(u, v) \leq w(x, y) $$

Por lo tanto:

$$ w(T') = w(T) - w(x, y) + w(u, v) \leq w(T) $$

Pero ya que $T$ es un MST, se debe dar que $w(T) \leq w(T')$, por lo que llegamos a la conclusión de que $T'$ es un MST también.

Finalmente, probemos que $(u, v)$ es segura para $A$. Notemos que $A \subseteq T$ y ya que $(x, y) \not in A$, tendremos que $A \subseteq T'$, por lo que $A \cup \{(u, v)\} \subseteq T'$ : $(u, v)$ es segura para $A$.

**Corolario:**

Sea un grafo no dirigido, conexo y ponderado $G = (V, E)$ con función de peso $w : E \to \mathbb{R}$. Sea $A$ un subconjunto de $E$ que está incluido en algún MST de $G$ y sea $C = (V_{C}, E_{C})$ una componente conexa en el bosque $G_{A} = (V, A)$. Si $(u, v)$ es una arista ligera que conecta a $C$ con alguna otra componente de $G_{A}$, entonces $(u, v)$ es segura para $A$. 

**Prueba:**

El corte $(V_{C}, V\backslash V_{C})$ respeta a $A$ y $(u, v)$ es una arista ligera para dicho corte, por lo que la arista $(u, v)$ es segura para $A$.

# Algoritmo de Kruskal

El algoritmo de Kruskal considera la arista de menor peso que aún no haya sido procesada como posible candidata a safe edge para el conjunto $A$. Notemos que si $(u, v)$ es dicha arista, solamente hay 2 opciones:

1. $u$ y $v$ están en diferentes componentes de $G_{A}$, en cuyo caso $(u, v)$ cruza el corte $(A, V\backslash A)$ y por consecuencia $(u, v)$ es safe edge para $A$.

2. $u$ y $v$ están en la misma componente de $G_{A}$, en cuyo caso no deberemos realizar ninguna acción, ya que al agregar la arista se formaría un ciclo en el MST procesado y dado que las aristas de $A$ tienen peso menor o igual al de $(u, v)$, por lo que $(u, v)$ no podría formar parte de un MST con dichos elementos.

```Python
Kruskal():
    A = {}
    for v in V:
        create-component(v)
    ordenar G.E por peso de arista no decreciente
    for (u, v) in G.E:
        if component(u) != component(v):
            A = A U {(u, v)}
            join-components(u, v)
    return A
```

De esta forma, asumiendo que tenemos una estructura de datos (*Disjoint Set Union*, que se verá más adelante y por ahora solo usaremos la estructura de datos como caja negra) que nos permita verificar en $O(\log{V})$ si dos nodos pertenecen a la misma componente y también unir dos componentes disjuntas en la misma complejidad, obtendremos una complejidad final de $O(E\log{E})$ por ordenar las aristas y $O(E\log{V})$. Ya que la cantidad de aristas $E = O(V^{2})$, tendremos $O(E\log{V})$.

## Problemas para implementar

- [Minimum Spanning Tree](https://www.spoj.com/problems/MST/)

# Algoritmo de Prim

El algoritmo de Prim considera una raiz arbitraria $r$ desde la cual se expandirá el MST agregando aristas seguras una por una. La idea para obtener una arista ligera es considerar los vértices $v$ que no están en $G_{A}$, ordenarlos por el mínimo peso de alguna arista $(u, v)$ tal que $u \in A$ y usar la mínima de todas esas aristas.

Para ordenar los nodos que **no pertenecen** a $V_{A}$ se usa una cola de prioridades para mínimo $Q$ basada en un atributo $weight[v]$, el cual estará definido para cada vértice como el mínimo peso de todas las aristas que conectan el árbol $G_{A}$ al nodo en cuestión. Por convención, se asume $weight = \infty$ si no existe ninguna arista en dicho momento. También usaremos un atributo $padre[v]$, que denotará el padre de $v$ en el árbol. 

Este algoritmo obtendrá $padre[v]$ para todo nodo $V \backslash \{s\}$ y el conjunto $A$ estará conformado por las aristas de manera implícita considerando los nodos en la cola $Q$:

$$ A = \{(padre[v], v) : v \in V \backslash \{s\} \backslash Q\} $$

Y al terminar la cola $Q$ estará vacía, por lo que nos quedaremos con el MST:

$$ A = \{(padre[v], v) : v \in V \backslash \{s\}\} $$

El algoritmo original es el siguiente:

```Python
Prim(r):
    for v in V:
        padre[v] = NULL
        weight[v] = inf
    weight[r] = 0
    priority_queue Q = V
    while Q not empty:
        u = Q.top() # Extrae el v con minimo weight
        Q.pop()
        for (u, v) in E:
            if v in Q and w(u,v) < weight[v]:
                weight[v] = w(u, v)
                padre[v] = u
    A = {}
    for v in V:
        if v == r: continue
        A = A U {(padre[v], v)}
    return A
```

El algoritmo anterior tendrá una complejidad de $O(E\log{V})$, ya que por cada arista realizaremos modificaciones en la cola de prioridades, lo cual toma $O(\log{V})$ por vez. Si se usa Fibonacci Heap, la complejidad se reduce a $O(E + V\log{V})$.

Una alternativa al algoritmo anterior es plantearlo como un algoritmo de Dijkstra, solo que en vez de comparar las distancias, compararemos los pesos de las aristas con las que llegó al nodo:

```Python
Prim(r):
    for v in V:
        padre[v] = NULL
        weight[v] = inf
    weight[r] = 0
    priority_queue Q
    Q.push({0, s})
    while Q not empty:
        u = Q.top().second # Extrae el v con minimo weight
        we = Q.top().first
        Q.pop()
        if weight[u] < we: continue
        for v in G[u]:
            if w(u,v) < weight[v]:
                weight[v] = w(u, v)
                padre[v] = u
                Q.push({weight[v], v})
    A = {}
    for v in V:
        if v == r: continue
        A = A U {(padre[v], v)}
    return A
```

El algoritmo anterior también tiene complejidad $O(E\log{V})$. Para grafos densos, podemos considerar una variación en $O(V^{2})$ al igual que con el algoritmo de Dijkstra, usando el arreglo de booleanos `vis` de apoyo:

```Python
Prim(r):
    for v in V:
        padre[v] = NULL
        weight[v] = inf
        vis[v] = False
    weight[r] = 0
    for i = 1 to n - 1:
        best = NULL
        for v in V:
            if vis[v]: continue
            if best == NULL or weight[v] < weight[best]:
                best = v;
        if best == NULL: break
        vis[best] = True
        for v in V:
            if vis[v]: continue
            if d[best][v] < weight[v]:
                weight[v] = d[best][v]
                padre[v] = best
    A = {}
    for v in V:
        if v == r: continue
        A = A U {(padre[v], v)}
    return A
```

## Problemas para implementar

- [Build The Road Network](https://vjudge.net/problem/Gym-270304E)

# Algoritmo de Boruvka

El algoritmo de Boruvka es uno poco recurrente en los problemas, pero muy útil ya que es poco probable que un problema que se resuelve con este algoritmo se pueda resolver con los dos anteriores. Lo que plantea es mantener un bosque (*forest*) de componentes, luego hallamos aristas ligeras para cada componente y las agregamos al conjunto de aristas.

Este proceso unirá las componentes y, en el peor de los casos, dividirá la cantidad de componentes $m$ entre 2 cada iteración, por lo que realizará $O(\log{V})$ iteraciones.

```Python
Boruvka():
    F = {V} # Bosque de componentes inicial, todos los nodos aislados
    while |F| > 1:
        Hallar las componentes conexas de F (conjunto C) y etiquetar cada nodo con su componente
        for c in C:
            weight[c] = inf
            edge[c] = NULL
        for (u, v) in E:
            if comp[u] != comp[v]: # Estan en diferentes componentes
                if w(u, v) < weight[comp[u]]:
                    weight[comp[u]] = w(u, v)
                    edge[comp[u]] = (u, v)
                if w(u, v) < weight[comp[v]]:
                    weight[comp[v]] = w(u, v)
                    edge[comp[v]] = (u, v)
        for c in C:
            if edge[c] == NULL: continue
            Agregar la arista edge[c] a F
    return F
```

Notemos que el trabajo por cada iteración se reduce a $O(E)$, y si mantenemos las aristas en un `std::set`, podremos obtener una complejidad de $O(E\log^{2}{v})$. Una optimización se lograría usando DSU $O(\alpha(V))$ para reducir la complejidad a $O(E\log{V}\alpha(V))$, que en términos prácticos tendría un rendimiento similar a $O(E\log{V})$.

## Problemas para implementar

- [Connecting the Graph](https://csacademy.com/contest/fii-code-2020-round-2/task/connecting-the-graph/)

# Problemas para practicar

- [Cost](https://www.spoj.com/problems/KOICOST/)
- [BMW](https://www.spoj.com/problems/MARYBMW/)
- [Roads of NITT](https://www.spoj.com/problems/NITTROAD/)
- [Hongcow Builds A Nation](https://codeforces.com/contest/744/problem/A)
- [Petya and the Road Repairs](https://www.spoj.com/problems/IITWPC4I/)
- [Fullmetal Alchemist](https://www.codechef.com/ICL2016/problems/ICL16A)
- [MST Queries](https://www.codechef.com/problems/MSTQS/)